<a href="https://colab.research.google.com/github/Kahiro-M/pyAanalys/blob/main/%E7%B6%99%E7%B6%9A%E7%8E%87%E9%9B%86%E8%A8%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ドライブマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
# ファイルコピー
!cp "/content/drive/MyDrive/Colab Notebooks/MOCK_DATA2.csv" "/content/data.csv"

In [62]:
# インポート
import pandas as pd
import datetime as dt

In [63]:
def get_month_delta(t1,t2):
    from datetime import datetime as dt
    print(type(t1))
    print(t1)
    print(type(t2))
    print(t2)
    delta = abs( t1.year - t2.year )*12 + abs( t1.month - t2.month )
    return delta

In [64]:
# ファイル読み込み
file_path = 'data.csv'
# df = pd.read_csv(file_path, encoding='cp932')
df = pd.read_csv(file_path)

# 日付データ形式に変換
df['date'] = pd.to_datetime(df['date'])

# ID、日付でソート
df = df.sort_values(['id','date'])

# 日付から年月日を取得
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['year-month'] = df['date'].dt.strftime('%Y-%m')

In [65]:
# データ確認
print(df.query('id == 100'))

     id       date  price  year  month  day year-month
0   100 2021-01-20   9153  2021      1   20    2021-01
1   100 2021-02-20   7344  2021      2   20    2021-02
2   100 2021-03-20  13844  2021      3   20    2021-03
3   100 2021-04-20   6513  2021      4   20    2021-04
4   100 2021-05-20  11346  2021      5   20    2021-05
5   100 2021-06-20  12139  2021      6   20    2021-06
6   100 2021-07-20  13864  2021      7   20    2021-07
7   100 2021-08-20  10271  2021      8   20    2021-08
8   100 2021-09-20   8497  2021      9   20    2021-09
9   100 2021-10-20  11718  2021     10   20    2021-10
10  100 2021-11-20   9286  2021     11   20    2021-11
11  100 2021-12-20   5534  2021     12   20    2021-12
12  100 2022-01-20  15219  2022      1   20    2022-01
13  100 2022-02-20  13938  2022      2   20    2022-02
14  100 2022-03-20  12640  2022      3   20    2022-03
15  100 2022-04-20   7881  2022      4   20    2022-04
16  100 2022-05-20   8162  2022      5   20    2022-05
17  100 20

In [66]:
df.groupby(['id','year']).count()

date  price  month  day  year-month
id  year                                     
100 2021    12     12     12   12          12
    2022    12     12     12   12          12
    2023    12     12     12   12          12
101 2021     6      6      6    6           6
    2022     6      6      6    6           6
    2023     6      6      6    6           6
102 2021     9      9      9    9           9
    2022     8      8      8    8           8
    2023     8      8      8    8           8
103 2021     3      3      3    3           3
    2022     3      3      3    3           3
    2023     6      6      6    6           6
104 2022     6      6      6    6           6
105 2023     6      6      6    6           6

In [67]:
# IDごとにグループ化
grouped_id = df.groupby(['id'])

# グループごとに月数を算出
result = pd.DataFrame()
for group_name, group_data in grouped_id:
    # 1個前のデータのカラムを用意
    group_data['prev_date'] = group_data['date'].shift(1)

    # 日付の間隔を算出
    group_data['day_diff'] = (group_data['date'] - group_data['prev_date'])
    group_data['day_diff_int'] = group_data['day_diff']/dt.timedelta(days=1)
    group_data['month_diff_int'] = (group_data['date'].dt.year-group_data['prev_date'].dt.year)*12 + (group_data['date'].dt.month-group_data['prev_date'].dt.month)

    # 継続期間(term)を算出　※5ヶ月差以上に変更点として1を設定し、term月差を0、term月差の累計をtermとする
    group_data['term_point'] = 0
    group_data['term_point'] = group_data['term_point'].where(group_data['month_diff_int']<5,1)
    group_data['term_month_diff'] = group_data['month_diff_int'].where(group_data['term_point']<1,0)
    group_data['term'] = group_data['term_point'].cumsum()

    # termごとの月差を累積（継続している月数）
    group_data['amount_term_month_diff'] = group_data.groupby(['term'])['term_month_diff'].cumsum()

    # 結果をDataFrameとして結合
    result = pd.concat([result, group_data], axis=0)

result.query('id == 105')

<ipython-input-67-79bb1ff33f6d>:6: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for group_name, group_data in grouped_id:


,id,date,price,year,month,day,year-month,prev_date,day_diff,day_diff_int,month_diff_int,term_point,term_month_diff,term,amount_term_month_diff
97,105,2023-01-10,9153,2023,1,10,2023-01,NaT,NaT,NaN,NaN,1,0.0,1,0.0
98,105,2023-03-10,13844,2023,3,10,2023-03,2023-01-10,59 days,59.0,2.0,0,2.0,1,2.0
99,105,2023-05-10,6513,2023,5,10,2023-05,2023-03-10,61 days,61.0,2.0,0,2.0,1,4.0
100,105,2023-07-10,11346,2023,7,10,2023-07,2023-05-10,61 days,61.0,2.0,0,2.0,1,6.0
101,105,2023-09-10,12139,2023,9,10,2023-09,2023-07-10,62 days,62.0,2.0,0,2.0,1,8.0
102,105,2023-11-10,13864,2023,11,10,2023-11,2023-09-10,61 days,61.0,2.0,0,2.0,1,10.0


In [70]:
# print(term_diff.query('id == 100'))
# print(term_diff.query('id == 101'))

# 日付と継続期間の情報のみ抜粋
term_diff = result[['id','date','year','month','term','amount_term_month_diff']]
# term_diff = result


start_date = "2021-01-01"
end_date = "2023-12-31"

# print(term_diff.query('term == 1').groupby(['id']).last().reset_index())
# print(term_diff.query('date <= @start_date').groupby(['id']).last().reset_index())
# print(term_diff.query('@start_date <= date <= @end_date'))
# print(term_diff.query('@end_date <= date').groupby(['id']).first().reset_index())


# 指定範囲直前のデータと指定範囲のデータを統計対象データとする
target_data = pd.concat([
    # term_diff.query('date <= @start_date').groupby(['id']).last().reset_index(),
    # term_diff.query('100 <= id <= 103 and @start_date <= date <= @end_date'),
    term_diff.query('@start_date <= date <= @end_date'),
    # term_diff.query('@end_date <= date').groupby(['id']).first().reset_index(),
    # term_diff.query('@start_date <= date <= @end_date').groupby(['id']).last().reset_index(),
    ],axis=0)
# target_data = term_diff.query('@start_date <= date <= @end_date')

count_table = target_data.pivot_table(
    index=['amount_term_month_diff'],
    columns=['year'],
    values=['id'],
    aggfunc='count',
)

count_table.to_csv(start_date+'_to_'+end_date+'_output.csv')

# print(count_table)
print(count_table.query('amount_term_month_diff > 0').sum())
print(count_table.query('amount_term_month_diff > 6').sum())
print(count_table.query('amount_term_month_diff > 12').sum())


    year
id  2021    26.0
    2022    34.0
    2023    37.0
dtype: float64
    year
id  2021    11.0
    2022    31.0
    2023    34.0
dtype: float64
    year
id  2021     0.0
    2022    25.0
    2023    32.0
dtype: float64
